In [1]:
import os
import time
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2
import sys
print(pd.__version__)

from postgressql import *

1.3.3


In [2]:
conn_params_dic = {
    "host": "10.0.0.102",
    "dbname": "mastr",
    "user": "uba_user",
    "password": "UBAit2021!",
    "port": "5432"
}

sqlpath = "./sqlCommands/loadMastrDBfiles.sql"

def format_bytes(size):
    # 2**10 = 1024
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'kilo', 2: 'mega', 3: 'giga', 4: 'tera'}
    while size > power:
        size /= power
        n += 1
    return size, power_labels[n]+'bytes'

def build_postgres_conn_string (param:dict) -> str:
    """

    """
    return f'postgresql+psycopg2://{param["user"]}:{param["password"]}@{param["host"]}:{param["port"]}/{param["dbname"]}'

def create_postgres_engine (param:dict):
    """
    
    """
    conString = build_postgres_conn_string(param)
    engine = sqlalchemy.create_engine(conString, pool_recycle=3600)
    return engine

def read_postgres_from_queryfile (sqlpath:str, postgresLogin:dict):# -> pandas.core.frame.DataFrame:
    """

    """
    engine = create_postgres_engine(postgresLogin)
    
    scriptFile = open(sqlpath,'r')
    script = scriptFile.read()
    df = pd.read_sql(script, engine)

    return df

import updateMastr

In [8]:
sqlpath = "./sqlCommands/loadMastrDBfiles.sql"
df = read_postgres_from_queryfile(sqlpath, conn_params_dic)
df = df.dropna()
df.head()#.filter(regex="grad(?i)").head()

,Laengengrad,Breitengrad
8,10.078254,49.652734
47,12.622059,48.082270
63,12.206935,49.235901
69,12.347704,50.884458
119,8.487537,49.829086


In [9]:
print(f"Size: {format_bytes(sys.getsizeof(df))}")
print(f"Size: {format_bytes(df.memory_usage(index=True, deep=True).sum())}")

df.count()

Size: (4.0572662353515625, 'megabytes')
Size: (4.0572509765625, 'megabytes')


Laengengrad    177264
Breitengrad    177264
dtype: int64

# PLOTTING SPACIAL DATA

In [5]:
import geopandas as gp
import matplotlib.pyplot as plt
import fiona

ImportError: libpoppler.so.91: cannot open shared object file: No such file or directory

In [6]:
gdf = gp.GeoDataFrame(df, geometry=gp.points_from_xy(df.Laengengrad, df.Breitengrad))
world = gp.read_file(gp.datasets.get_path('naturalearth_lowres'))

# We restrict to South America.
ax = world[world.continent == 'Europe'].plot(
    color='white', edgecolor='black')

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color='red')

plt.show()

ImportError: the 'read_file' function requires the 'fiona' package, but it is not installed or does not import correctly.
Importing fiona resulted in: libpoppler.so.91: cannot open shared object file: No such file or directory